In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch 
import pandas as pd
from sklearn.model_selection import train_test_split
import re

In [18]:
data_bancodobrasil = pd.read_json("data/banco_brasil.json")
data_bancodobrasil_con = pd.DataFrame({'texto': data_bancodobrasil["title"] + " " + data_bancodobrasil["description"]})

data_magalu = pd.read_json("data/magazineluiza.json")
data_magalu_con = pd.DataFrame({'texto': data_magalu["title"] + " " + data_magalu["description"]})


data_mglu3 = pd.read_json("data/mglu3.json")
data_mglu3_con = pd.DataFrame({'texto': data_mglu3["title"] + " " + data_mglu3["description"]})


data_varejo = pd.read_json("data/varejo.json")
data_varejo_con = pd.DataFrame({'texto': data_varejo["title"] + " " + data_varejo["description"]})

data_petro = pd.read_json('data/Petrobras_noticias.json')
data_petro_con = pd.DataFrame({'texto': data_petro["title"] + " " + data_petro["description"]})


data = pd.merge(data_bancodobrasil_con, data_magalu_con, on='texto', how='outer').merge(data_mglu3_con, on='texto', how='outer').merge(data_varejo_con, on='texto', how='outer').merge(data_petro_con, on='texto', how='outer')
data['texto'] = data['texto'].str.replace('\n', '')
data['texto'] = data['texto'].str.replace('\t', '')
data['texto'] = data['texto'].str.replace('"', '')
data['texto'] = data['texto'].str.replace("'", '')
data['texto'] = data['texto'].str.replace('/', '')





In [19]:
tokenizer = AutoTokenizer.from_pretrained("mrm8488/deberta-v3-ft-financial-news-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/deberta-v3-ft-financial-news-sentiment-analysis")



In [20]:
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1).item()
    return predicted_class, probabilities


In [22]:
data['sentiment'] = data['texto'].apply(lambda x: predict_sentiment(x)[0])
data['probabilities'] = data['texto'].apply(lambda x: predict_sentiment(x)[1].tolist())

data.to_csv('output_deberta.csv')